## Using the FINBOURNE Python SDKs

We provide many tools for interacting with FINBOURNE products in Python and Jupyter.

The FINBOURNE Python SDKs provides a set of objects and functions which enable you to call our APIs without having to write any direct http or serialisation code. We also provide other manually-written packages which make interacting with LUSID programmatically easier.
In this section we'll:
- Introduce the FINBOURNE Python SDKs, describing how they are used.
- Describe some of the tools for interacting with LUSID in our hosted Jupyter environment.

### Authentication
We have a set of SDKs such as the lusid-sdk-python, drive-sdk-python and finbourne-access-sdk-python, all hosted on Pypi. These SDKs provide api objects, with methods that can be used to perform actions in LUSID.

All authenticated calls to the LUSID API require an OpenID Connect ID token which is issued from your token issuer url. The details of these can be found on your LUSID portal under "Applications" within the "Identity and Access Management" section.

This is explained in greater detail in our [knowledge base.](https://support.lusid.com/knowledgebase/article/KA-01667/en-us)

In [1]:
import lusid
import os
from lusidjam import RefreshingToken
from lusid.utilities import ApiConfigurationLoader
import fbnsdkutilities.utilities as utils

try:
    # Authenticate to SDK
    # Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
    secrets_path = os.getenv("FBN_SECRETS_PATH")
    # Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
    if secrets_path is None:
        secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

    api_factory = utils.ApiClientFactory(
        lusid,
        token = RefreshingToken(), 
        api_secrets_filename = secrets_path,
        app_name = "LusidJupyterNotebook"
    )

    print([api for api in dir(lusid.api) if "Api" in api])
except Exception as e:
    print(e)



['AborApi', 'AborConfigurationApi', 'AggregationApi', 'AllocationServiceApi', 'AllocationsApi', 'ApplicationMetadataApi', 'BlocksApi', 'CalendarsApi', 'ChartOfAccountsApi', 'ComplexMarketDataApi', 'ComplianceApi', 'ConfigurationRecipeApi', 'ConventionsApi', 'CorporateActionSourcesApi', 'CounterpartiesApi', 'CustomEntitiesApi', 'CustomEntityDefinitionsApi', 'CutLabelDefinitionsApi', 'DataTypesApi', 'DerivedTransactionPortfoliosApi', 'EntitiesApi', 'ExecutionsApi', 'InstrumentEventsApi', 'InstrumentsApi', 'LegalEntitiesApi', 'OrderGraphApi', 'OrderInstructionsApi', 'OrdersApi', 'PackagesApi', 'ParticipationsApi', 'PersonsApi', 'PlacementsApi', 'PortfolioGroupsApi', 'PortfoliosApi', 'PropertyDefinitionsApi', 'QuotesApi', 'ReconciliationsApi', 'ReferencePortfolioApi', 'RelationDefinitionsApi', 'RelationsApi', 'RelationshipDefinitionsApi', 'RelationshipsApi', 'SchemasApi', 'ScopesApi', 'SearchApi', 'SequencesApi', 'StructuredResultDataApi', 'SystemConfigurationApi', 'TaxRuleSetsApi', 'Trans

Here's an example using the lusid python sdk. We initialise an `api_factory` using our access details, which can be provided in either a secrets json file, or as environment variables. Here we have some environment variables set, we use `RefreshingToken` to grab a token using these environment variables.

We then print a list of the available APIs in the lusid preview sdk.



In [2]:
try:
    api_instance = api_factory.build(lusid.ApplicationMetadataApi)
    # GetLusidVersions: Get LUSID versions
    api_response = api_instance.get_lusid_versions()
    print(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling ApplicationMetadataApi->get_lusid_versions: %s\n" % e)

{'api_version': 'v0',
 'build_version': '0.6.11269.0',
 'excel_version': '0.5.3236',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-ci.lusid.com/app/insights/logs/0HMQGHN9BNJGO:0000011E',
            'method': 'GET',
            'relation': 'RequestLogs'}]}


Here, we use our `api_factory` to build an `ApplicationMetadatApi` object, which we can use to send requests for our ApplicationMetadata REST endpoints.
We then call the `get_lusid_versions` method on this object, which sends a request to the ApplicationMetadataApi->get_lusid_versions endpoint, returning the result as a json string.

### Tools for interacting with LUSID in our hosted Jupyter environment.
We provide a Jupyter environment that you can access and use to interactively write Python and dotnet scripts. In our Jupyter environment, the preview SDKs come installed by default.

We provide the lusidjam library, which can be used to provide an authentication token without re-entering credentials into your Python scripts:

In [3]:
import lusidjam

api_factory = utils.ApiClientFactory(
    lusid,
    token = lusidjam.RefreshingToken()
)

Here we've built an api-factory using credentials stored in our jupyter environment.

We also provide a custom magic command to query luminesce:

In [4]:
import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client
from lusidjam import RefreshingToken

token = RefreshingToken()
lusid_api_url = api_factory.api_client.configuration.host
lumi_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "honeycomb"
os.environ["FBN_LUMI_API_URL"] = lumi_api_url
lumi_url = os.getenv("FBN_LUMI_API_URL")

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    lm_client = Client(token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df

# In an interactive session, we need to delete to avoid
# name conflicts for automagic to work on line magics.
del luminesce

In [5]:
%%luminesce
SELECT * FROM Lusid.Instrument.Equity LIMIT 10

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Isin,Sedol,Cusip,Ticker,...,CountryOfIncorporation,Exchange,IssuerId,IssuerName,LegalEntityId,NaicsCode,ParValue,PrimaryCurrency,PrimaryExchange,SecurityId
0,LUID_0001E0IN,BBG00WGHTKZ1186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_0001E0ML,BBG00WGHTKZ1376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_0001E0J7,BBG00WGHTKZ1377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUID_0001E1IP,BBG00WGHTKZ1929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LUID_0001E0X7,BBG00WGHTKZ1595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,LUID_0001E0JE,BBG00WGHTKZ1100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LUID_0001E11Y,BBG00WGHTKZ1529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LUID_0001E0LM,BBG00WGHTKZ1006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LUID_0001E0W6,BBG00WGHTKZ1716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LUID_0001E1CK,BBG00WGHTKZ2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This cell magic runs any statement in the cell below the magic command, displaying a pandas dataframe containing the output of the luminesce query.

In [6]:
results = %luminesce SELECT * FROM Lusid.Instrument.Equity LIMIT 10
results.head()

,LusidInstrumentId,Figi,ClientInternal,QuotePermId,EdiKey,SecurityNumber,Isin,Sedol,Cusip,Ticker,...,CountryOfIncorporation,Exchange,IssuerId,IssuerName,LegalEntityId,NaicsCode,ParValue,PrimaryCurrency,PrimaryExchange,SecurityId
0,LUID_00010Q97,NaN,Equity5068e3d3-adc6-4322-97b5-fe484e791221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUID_0000Y3B0,NaN,Equitydf5f6ba0-fb3a-4f3f-83de-a498f7056648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUID_000140Z8,NaN,Equityf3d2e4fc-2898-43f0-a147-4566184df03a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUID_00013KNX,NaN,Equity800ed431-7be6-4992-8643-74d2e6335572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LUID_0000YB9E,NaN,Equity5346b78e-2aa9-43dd-8b3c-57e282aa0e80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This line magic runs the query on the same line after the magic command, allowing us to use the result of the query in our Python code.